In [2]:
import pandas as pd
import requests
from datetime import datetime
import time

# Start with fewer coins for testing
coin_id_to_symbol = {
      'aave': 'AAVE',
      'binancecoin': 'BNB',
      'cosmos': 'ATOM',
      'dogecoin': 'DOGE',
      'ethereum': 'ETH',
      'litecoin': 'LTC',
      'nem': 'XEM',
      'stellar': 'XLM',
      'uniswap': 'UNI',
      'wrapped-bitcoin': 'WBTC'      # Add one more for testing
}

headers = {
    "accept": "application/json",
    "x-cg-demo-api-key": "CG-xnM7ZM5eABLu7AZ4iWKApvmx" # Replace with your API KEY
}

market_url = "https://api.coingecko.com/api/v3/coins/{id}/market_chart"
ohlc_url = "https://api.coingecko.com/api/v3/coins/{id}/ohlc"

# Start with fewer days for testing
params = {
    "vs_currency": "usd",
    "days": "365",  # Start with 30 days for testing
    "interval": "daily"
}

# Test API connection first
print("🔍 Testing API connection...")
test_url = "https://api.coingecko.com/api/v3/ping"
try:
    test_response = requests.get(test_url, headers=headers)
    print(f"API Ping Status: {test_response.status_code}")
    if test_response.status_code == 200:
        print("✅ API is reachable")
    else:
        print(f"❌ API ping failed: {test_response.text}")
except Exception as e:
    print(f"❌ Network error: {e}")

print("\n" + "="*50)

# Final results
all_data = []

for coin_id, symbol in coin_id_to_symbol.items():
    print(f"\n📈 Fetching data for {symbol} ({coin_id})...")
    
    # Add delay to avoid rate limiting
    time.sleep(1)

    # --- Fetch market data (price, market cap, volume)
    try:
        market_response = requests.get(market_url.format(id=coin_id), headers=headers, params=params)
        print(f"Market data status: {market_response.status_code}")
        
        if market_response.status_code != 200:
            print(f"Market data error: {market_response.text}")
            continue
            
    except Exception as e:
        print(f"Market data request failed: {e}")
        continue

    # Add another delay
    time.sleep(1)
    
    try:
        ohlc_response = requests.get(ohlc_url.format(id=coin_id), headers=headers, params={"vs_currency": "usd", "days": "30"})
        print(f"OHLC data status: {ohlc_response.status_code}")
        
        if ohlc_response.status_code != 200:
            print(f"OHLC data error: {ohlc_response.text}")
            # Continue without OHLC data
            ohlc_data = []
        else:
            ohlc_data = ohlc_response.json()
            
    except Exception as e:
        print(f"OHLC data request failed: {e}")
        ohlc_data = []

    if market_response.status_code == 200:
        market_data = market_response.json()

        prices = market_data.get("prices", [])
        market_caps = market_data.get("market_caps", [])
        total_volumes = market_data.get("total_volumes", [])
        
        print(f"Found {len(prices)} price points")

        # Create a mapping from date -> OHLC
        ohlc_by_date = {}
        for entry in ohlc_data:
            dt = datetime.fromtimestamp(entry[0] / 1000).strftime("%Y-%m-%d")
            ohlc_by_date[dt] = {
                "open": round(entry[1], 2),
                "high": round(entry[2], 2),
                "low": round(entry[3], 2),
                "close": round(entry[4], 2),
            }

        # Process the data
        for i in range(len(prices)):
            timestamp = prices[i][0]
            dt = datetime.fromtimestamp(timestamp / 1000).strftime("%Y-%m-%d")

            entry = {
                "symbol": symbol,
                "date": dt,
                "price_usd": round(prices[i][1], 2),
                "market_cap_usd": round(market_caps[i][1], 2) if i < len(market_caps) else None,
                "total_volume_usd": round(total_volumes[i][1], 2) if i < len(total_volumes) else None,
                "open": ohlc_by_date.get(dt, {}).get("open"),
                "high": ohlc_by_date.get(dt, {}).get("high"),
                "low": ohlc_by_date.get(dt, {}).get("low"),
                "close": ohlc_by_date.get(dt, {}).get("close")
            }

            all_data.append(entry)
        
        print(f"✅ Successfully processed {len(prices)} records for {symbol}")
    else:
        print(f"❌ Failed for {symbol}: {market_response.status_code}")

print(f"\n" + "="*50)
print(f"🎯 Total records collected: {len(all_data)}")

# Check if we have any data
if not all_data:
    print("\n❌ No data was collected. Possible issues:")
    print("1. API key might be invalid or expired")
    print("2. Rate limiting - try with fewer coins or longer delays")
    print("3. Network connectivity issues")
    print("4. API endpoint changes")
    print("\n💡 Troubleshooting steps:")
    print("- Try getting a new API key from CoinGecko")
    print("- Test with just 1 cryptocurrency first")
    print("- Check if you've exceeded your API quota")
else:
    # Preview the first 5 rows
    print(f"\n✅ Successfully collected {len(all_data)} records!")
    print("\nSample Output:")
    for i, row in enumerate(all_data[:5]):
        print(f"{i+1}. {row}")

    # Convert the list to a DataFrame
    df = pd.DataFrame(all_data)

    # Display DataFrame info
    print(f"\n📊 DataFrame Info:")
    print(f"Total records: {len(df)}")
    print(f"Data shape: {df.shape}")
    print(f"Date range: {df['date'].min()} to {df['date'].max()}")

    if 'symbol' in df.columns:
        print(f"🪙 Cryptocurrencies: {df['symbol'].nunique()}")
        print(f"Symbols: {df['symbol'].unique().tolist()}")

    # Save the data
    try:
        # Try saving locally first to test
        df.to_csv('crypto_test_data.csv', index=False)
        print(f"\n📁 Data saved locally as 'crypto_test_data.csv'")
        
        # Now try lakehouse
        csv_output_path = 'abfss://Group1_CyrptoIntelligence@onelake.dfs.fabric.microsoft.com/Group1CryptoProjectLakehouse.Lakehouse/Files/crypto_historical_data/historicalcryptodata.csv'
        df.to_csv(csv_output_path, index=False)
        print(f"📁 Data also saved to lakehouse: {csv_output_path}")
        
    except Exception as e:
        print(f"\n📁 Data saved locally as 'crypto_test_data.csv'")
        print(f"❌ Lakehouse save failed: {e}")

StatementMeta(, a5388d2b-c377-483e-93b6-21ddd976c23e, 4, Finished, Available, Finished)

🔍 Testing API connection...
API Ping Status: 200
✅ API is reachable


📈 Fetching data for AAVE (aave)...
Market data status: 200
OHLC data status: 200
Found 366 price points
✅ Successfully processed 366 records for AAVE

📈 Fetching data for BNB (binancecoin)...
Market data status: 200
OHLC data status: 200
Found 366 price points
✅ Successfully processed 366 records for BNB

📈 Fetching data for ATOM (cosmos)...
Market data status: 200
OHLC data status: 200
Found 366 price points
✅ Successfully processed 366 records for ATOM

📈 Fetching data for DOGE (dogecoin)...
Market data status: 200
OHLC data status: 200
Found 366 price points
✅ Successfully processed 366 records for DOGE

📈 Fetching data for ETH (ethereum)...
Market data status: 200
OHLC data status: 200
Found 366 price points
✅ Successfully processed 366 records for ETH

📈 Fetching data for LTC (litecoin)...
Market data status: 200
OHLC data status: 200
Found 366 price points
✅ Successfully processed 366 records for LTC

📈 Fetchin